In [30]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search
import typing
from keys import API_KEY


# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-4"
openai.api_key = API_KEY


In [8]:
# download pre-chunked text and pre-computed embeddings
# this file is ~200 MB, so may take a minute depending on your connection speed
embeddings_path = r"C:\Users\aliyu\OneDrive\Documents\AI\Projects\petrobot\petrel_manual.csv"

df = pd.read_csv(embeddings_path)
df = df[['text','embedding']]

In [11]:
df.head()

,text,embedding
0,\nWWeellccoommee ttoo tthhee PPeettrreell** hh...,"[-0.0036606036592274904, -0.009110068902373314..."
1,Petrel Exploration Geophysics \nInterpret regi...,"[-0.024511124938726425, 0.005252866540104151, ..."
2,Imaging | Petrel Fault Analysis | Petrel Well ...,"[-0.024689018726348877, -0.005831622518599033,..."
3,Framework \nPlug-ins for Petrel\nLeverage pred...,"[0.005327986553311348, -0.0005350260762497783,..."
4,Petrel Geophysics \nPerform rapid 2D and 3D se...,"[-0.03281170502305031, 0.0006662339437752962, ..."


In [27]:
# convert embeddings from CSV str type back to list type
df['embedding'] = df['embedding'].apply(ast.literal_eval)

In [28]:
# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100) -> typing.Tuple[typing.List[str], typing.List[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [31]:
# examples
strings, relatednesses = strings_ranked_by_relatedness("dynamic model", df, top_n=50)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.806


'2. Surfaces \n3. Property models'

relatedness=0.800


'Figure 5 \nThe impact on the model becomes much clearer when a larger range is used for the modeling'

relatedness=0.793


...

'Optimization \nRapidly update reservoir models, better manage risk, and easily share\nknowledge\nThe Workflow Editor, an integral part of Petrel software, captures data parameters and\nrelationships that enable rapid updating of reservoir models as information from new wells\narrives.\nGeoscientists and engineers can create multiple model realizations to assess the impact on\nreserve volumetrics or to cost well placement. Engineers can run all the possible scenarios in\nECLIPSE directly, without leaving Petrel.\nIn addition to understanding uncertainty and risk management, the Workflow Editor empowers\nknowledge sharing, allowing best practices and workflows to be easily shared across your\norganization.\nWith Petrel, asset teams can reduce project cycle time and maximize productivity.\nUncertainty and Optimization\nThe Uncertainty and Optimization process allows you to to sensitivity analysis. You can create\nproxies for volumetric or simulation cases using experimental designs for f

relatedness=0.769


'Views \nthe views in here allow you to choose different ways to visualize the simulation data.\nDynamic data allows you to plot simulation data against time or other simulation data. Case\nvariables allows you to plot simulation data at a given time against values of case variables.\nDynamic data\nThis is where you select simulation results to be displayed. This sub-tree is used by both of\nthe view types.\nIdentifier, Source data type, Fluid identifiers, Aquifer type etc\nfolders of filters used to qualify the data to be visualized by one of the view types held in the\nViews folder. These sub-trees are used by both of the view types.\nCase variables\nthis folder is used only by the Case variables view, to select a variable to be displayed on X-\naxis.'

relatedness=0.769


'Depth Conversion Process \nOnce a velocity model has been created, it can be used to depth convert objects. Objects which\ncan be depth converted include:'

In [ ]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = 'Use the below articles on the 2022 Winter Olympics to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about the 2022 Winter Olympics."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message